In [1]:
import datetime as dt
import numpy as np
import pandas as pd

In [2]:
from Curves import Curves
from ImportData import importSWEiopa
from BondClasses import ZeroCouponBond,ZeroCouponBondPriced

In [3]:
paramfile = pd.read_csv("Parameters.csv")
paramfile.index = paramfile["Parameter"]
del paramfile["Parameter"]

In [4]:
spreadfile = pd.read_csv("Sector_Spread.csv")
spreadfile.index = spreadfile["NACE"]
del spreadfile["NACE"]

# IMPORT EIOPA CURVE

In [5]:
selected_param_file = paramfile.loc["EIOPA_param_file"][0]
selected_curves_file = paramfile.loc["EIOPA_curves_file"][0]
country = paramfile.loc["country"]["value"]
compounding = int(paramfile.loc["compounding"]["value"])

# Model parameters
MD = dt.datetime.strptime(paramfile.loc["Modelling_Date"]["value"],"%d/%m/%Y")
MD = dt.date(MD.year,MD.month,MD.day)

[maturities_country, curve_country, extra_param, Qb]= importSWEiopa(selected_param_file, selected_curves_file, country)

In [6]:
# Maturity of observations:
M_Obs = np.transpose(np.array(maturities_country.values))

# Ultimate froward rate ufr represents the rate to which the rate curve will converge as time increases:
ufr = extra_param.iloc[3]/100

# Convergence speed parameter alpha controls the speed at which the curve converges towards the ufr from the last liquid point:
alpha = extra_param.iloc[4]

# Qb calibration vector published by EIOPA for the curve calibration:
Qb = np.transpose(np.array(Qb.values))

# PRICING OF A COUPON BOND

In [7]:
bondfile = pd.read_csv("ZCB_Input.csv")
bondfile.index = bondfile["Asset ID"]
del bondfile["Asset ID"]
bondfile

,Asset_Type,NACE,Issue_Date,Maturity_Date,Notional_amount,Coupon_Rate,Frequency,Recovery_Rate,Default_Probability,Market_Price
Asset ID,,,,,,,,,,
1,Corporate_Bond,A1.4.5,3/12/2021,12/12/2026,100,0.03,1,0.4,0.03,94
2,Corporate_Bond,B5.2.0,3/12/2021,12/12/2028,100,0.05,1,0.4,0.03,92
3,Corporate_Bond,B8.9.3,3/12/2019,3/12/2020,100,0.04,1,0.4,0.03,96


In [8]:
def converttime(date_time):
    format = "%d/%m/%Y"
    datetime_str = dt.datetime.strptime(date_time,format).date()
    return datetime_str

In [9]:
nace = np.array([])
issuedate = np.array([])
maturitydate = np.array([])
notional = np.array([])
couponrate = np.array([])
frequency = np.array([])
recovrate = np.array([])
defprob = np.array([])
sspread = np.array([])
zspread = np.array([])
marketprice = np.array([])

for iCount in range(0,bondfile.index.size):
    nace = np.append(nace,bondfile["NACE"].iloc[iCount])
    issuedate = np.append(issuedate,converttime(bondfile["Issue_Date"].iloc[iCount]))
    maturitydate = np.append(maturitydate,converttime(bondfile["Maturity_Date"].iloc[iCount]))
    notional = np.append(notional,bondfile["Notional_amount"].iloc[iCount])
    couponrate = np.append(couponrate,bondfile["Coupon_Rate"].iloc[iCount])
    frequency = np.append(frequency,bondfile["Frequency"].iloc[iCount])
    recovrate = np.append(recovrate,bondfile["Recovery_Rate"].iloc[iCount])
    defprob = np.append(defprob,bondfile["Default_Probability"].iloc[iCount])
    sspread = np.append(sspread,spreadfile.loc[bondfile["NACE"].iloc[iCount]]["sSpread"])
    marketprice = np.append(marketprice, bondfile["Market_Price"].iloc[iCount])

In [10]:
# Example bond

#issuedate = np.array([dt.date(2021,12,3),dt.date(2021,12,3),dt.date(2019,12,3)])
#maturitydate = np.array([dt.date(2026,12,12),dt.date(2028,12,12),dt.date(2020,12,3)])
#notional = np.array([100,100,100])
#couponrate = np.array([0.03, 0.05,0.04])
#frequency = np.array([1, 1, 1])
#recovrate = np.array([0.4, 0.4,0.4]) 
#defprob =np.array([0.03, 0.03,0.04]) 
#sspread = np.array([0.008, 0.008,0.005])
#zspread = np.array([0.008, 0.008,0.005])

In [11]:
zcb = ZeroCouponBond(issuedate, maturitydate, frequency, notional, couponrate, recovrate, defprob, sspread, zspread, marketprice)

In [12]:
zcb.createcashflows()

In [13]:
zcbPriced = ZeroCouponBondPriced(MD,compounding)

In [14]:
[coupondatesconsidered,coupondatefrac] = zcbPriced.refactordates(zcb.coupondates,MD)
[notionaldatesconsidered,notionaldatefrac] = zcbPriced.refactordates(zcb.notionaldates,MD)

In [15]:
# carry forward the sspread of bonds that are still relevant
zcbPriced.sspread = []
zcbPriced.marketprice = []

for iCount in range(0,len(notionaldatesconsidered)-1):
    if notionaldatesconsidered[iCount] == [0]:
        zcbPriced.sspread = np.append(zcbPriced.sspread,zcb.sspread[iCount])
        zcbPriced.marketprice = np.append(zcbPriced.marketprice,zcb.marketprice[iCount])

In [16]:
# Numeric precision of the optimisation
Precision = float(paramfile.loc["Precision"][0])
# Targeted distance between the extrapolated curve and the ultimate forward rate at the convergence point
Tau = float(paramfile.loc["Tau"][0]) # 1 basis point
Country = paramfile.loc["country"]
InitialDate = paramfile.loc["Modelling_Date"]
Curves = Curves(ufr, Precision, Tau, InitialDate, Country)

In [17]:
coupontargetrates = []
notionaltargetrates = []

for iAsset in range(0,issuedate.size-1):

    if not notionaldatesconsidered:
       print("This bond has matured") 
    else: 
        coupontargetrates.append(Curves.SWExtrapolate(np.transpose(coupondatefrac[iAsset]),M_Obs, Qb, ufr, alpha))
        notionaltargetrates.append(Curves.SWExtrapolate(np.transpose(notionaldatefrac[iAsset]),M_Obs, Qb, ufr, alpha))
        zcbPriced.coupondatefrac.append(np.transpose(coupondatefrac[iAsset]))
        zcbPriced.notionaldatefrac.append(np.transpose(notionaldatefrac[iAsset]))
        zcbPriced.couponcfs.append(zcb.couponcfs[iAsset][coupondatesconsidered[iAsset]])
        zcbPriced.notionalcfs.append(zcb.notionalcfs[iAsset])

Remove cashflows that are not considered

In [18]:
#zcbPriced.PriceBond(coupontargetrates,notionaltargetrates,coupondatefrac,notionaldatefrac,zcbPriced.couponcfs,zcbPriced.notionalcfs,zcbPriced.sspread,np.array([0.01,0.01]))

### Calibration of two hypothetical bonds that mature after modelling date

In [19]:
def BisectionsSpread(zcbPriced, xStart, xEnd, coupontargetrate,notionaltargetrate,coupondatefrac,notionaldatefrac,couponcfs,notionalcf,sspread, marketprice, Precision, maxIter):
    """
    Bisection root finding algorithm for finding the root of a function. The function here is the allowed difference between the ultimate forward rate and the extrapolated curve using Smith & Wilson.

    Args:
        xStart =    1 x 1 floating number representing the minimum allowed value of the convergence speed parameter alpha. Ex. alpha = 0.05
        xEnd =      1 x 1 floating number representing the maximum allowed value of the convergence speed parameter alpha. Ex. alpha = 0.8
        M_Obs =     n x 1 ndarray of maturities of bonds, that have rates provided in input (r). Ex. u = [[1], [3]]
        r_Obs =     n x 1 ndarray of rates, for which you wish to calibrate the algorithm. Each rate belongs to an observable Zero-Coupon Bond with a known maturity. Ex. r = [[0.0024], [0.0034]]
        ufr  =      1 x 1 floating number, representing the ultimate forward rate. Ex. ufr = 0.042
        Tau =       1 x 1 floating number representing the allowed difference between ufr and actual curve. Ex. Tau = 0.00001
        Precision = 1 x 1 floating number representing the precision of the calculation. Higher the precision, more accurate the estimation of the root
        maxIter =   1 x 1 positive integer representing the maximum number of iterations allowed. This is to prevent an infinite loop in case the method does not converge to a solution         
    
    Returns:
        1 x 1 floating number representing the optimal value of the parameter alpha 

    Example of use:
        >>> import numpy as np
        >>> from SWCalibrate import SWCalibrate as SWCalibrate
        >>> M_Obs = np.transpose(np.array([1, 2, 4, 5, 6, 7]))
        >>> r_Obs =  np.transpose(np.array([0.01, 0.02, 0.03, 0.032, 0.035, 0.04]))
        >>> xStart = 0.05
        >>> xEnd = 0.5
        >>> maxIter = 1000
        >>> alfa = 0.15
        >>> ufr = 0.042
        >>> Precision = 0.0000000001
        >>> Tau = 0.0001
        >>> BisectionAlpha(xStart, xEnd, M_Obs, r_Obs, ufr, Tau, Precision, maxIter)
        [Out] 0.11549789285636511

    For more information see https://www.eiopa.europa.eu/sites/default/files/risk_free_interest_rate/12092019-technical_documentation.pdf and https://en.wikipedia.org/wiki/Bisection_method
    
    Implemented by Gregor Fabjan from Qnity Consultants on 17/12/2021.
    """   

    yStart = zcbPriced.OpenPriceBond(coupontargetrate,notionaltargetrate,coupondatefrac,notionaldatefrac,couponcfs,notionalcf,sspread,xStart)
    yEnd = zcbPriced.OpenPriceBond(coupontargetrate,notionaltargetrate,coupondatefrac,notionaldatefrac,couponcfs,notionalcf,sspread,xEnd)
    if np.abs(yStart-marketprice) < Precision:
        #self.alpha = xStart # If initial point already satisfies the conditions return start point
        return xStart
    if np.abs(yEnd-marketprice) < Precision:
        #self.alpha = xEnd
        return xEnd # If final point already satisfies the conditions return end point
    iIter = 0
    while iIter <= maxIter:
        xMid = (xEnd+xStart)/2 # calculate mid-point
        yMid = zcbPriced.OpenPriceBond(coupontargetrate,notionaltargetrate,coupondatefrac,notionaldatefrac,couponcfs,notionalcf,sspread,xMid) # What is the solution at midpoint
        if ((yStart-marketprice) == 0 or (xEnd-xStart)/2 < Precision): # Solution found
            #self.alpha = xMid
            return xMid
        else: # Solution not found
            iIter += 1
            if np.sign(yMid-marketprice) == np.sign(yStart-marketprice): # If the start point and the middle point have the same sign, then the root must be in the second half of the interval   
                xStart = xMid
            else: # If the start point and the middle point have a different sign than by mean value theorem the interval must contain at least one root
                xEnd = xMid
    return "Did not converge"

In [20]:
zcbPriced.zspread = []
for iCount in range(0,len(coupontargetrates)):
    zSpreadTmp = BisectionsSpread(zcbPriced, -0.2, 0.2, coupontargetrates[iCount],notionaltargetrates[iCount],zcbPriced.coupondatefrac[iCount],zcbPriced.notionaldatefrac[iCount],zcbPriced.couponcfs[iCount],zcbPriced.notionalcfs[iCount],zcbPriced.sspread[iCount],zcbPriced.marketprice[iCount], Precision, 1000)
    zcbPriced.zspread = np.append(zcbPriced.zspread,zSpreadTmp)

In [21]:
zcbPriced.marketprice

array([94., 92.])

In [22]:
coupontargetrates

[array([0.04474545, 0.04352242, 0.04174698, 0.04015462]),
 array([0.04474545, 0.04352242, 0.04174698, 0.04015462, 0.03903412,
        0.03830698])]

In [23]:
zcbPriced.sspread

array([0.01, 0.01])

In [24]:
zcbPriced.zspread

array([0.00056424, 0.02093499])